In [231]:
import time
import pickle

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup

import plotly.graph_objects as go

import pandas as pd

from datetime import datetime

In [232]:
# All_fight_data = [Events] Ordered from old -> new.
    # Event = [Fights, Date, Venue]
        # Fight = [Fighter1, Fighter2, Result]
            # Fighter1, Fighter2 = [Name, Total, Strikes]
                # Totals = [Total, First, Second, Third, Fourth, Fifth]
                    # Total, First, etc = [Knockdowns, Takedowns, Reversals, Submission Attempted, Control Time]
                        # Takedowns = [Landed, Attempted]
                # Strikes = [Total, First, Second, Third, Fourth, Fifth]
                    # Total, First, etc = [Significant Strikes, Head, Body, Leg, Distance, Clinch, Ground]
                        # Everything = [Landed, Attempted]
            # Result = [Winner, Method, Round, Time, Referee, Weight, Bonus, Details]

In [233]:
def open_data(name):
    with open(name, "rb") as fp:
        return pickle.load(fp)

In [234]:
def save_data(data, name):
    with open(name, "wb") as fp:
        pickle.dump(data, fp)

In [235]:
all_fight_data = open_data('all_fight_data')

In [236]:
def all_fighter_names(data):
    all_names = []
    for event in data:
        for fight in event[0]:
            if fight[0][0] not in all_names:
                all_names.append(fight[0][0])
                
            if fight[1][0] not in all_names:
                all_names.append(fight[1][0])
    return all_names

In [237]:
def all_referee_names(data):
    all_names = []
    for event in data:
        for fight in event[0]:
            if fight[2][4] not in all_names:
                all_names.append(fight[2][4])
    
    return all_names

In [238]:
def convert_date(date):
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    month = date.split(' ')[0]
    month = months.index(month) + 1
    
    day = int(date.split(',')[0].split(' ')[1])
    year = int(date.split(', ')[1])
    date = datetime(year, month, day)
    return date

In [239]:
def bar_data(data, group):
    # [name, ko, submission, decision, finish]
    if group == 'Fighters':
        return []
    else:
        all_r_names = all_referee_names(data)
        all_data = [[all_r_names[i], 0, 0, 0, 0] for i in range(len(all_r_names))]
        
        for event in data:
            for fight in event[0]:
                name_index = all_r_names.index(fight[2][4])
                    
                if fight[2][1] == 'KO/TKO':
                    all_data[name_index][1] += 1
                    all_data[name_index][4] += 1
                elif fight[2][1] == 'Submission':
                    all_data[name_index][2] += 1
                    all_data[name_index][4] += 1
                else:
                    all_data[name_index][3] += 1
                    all_data[name_index][4] -= 1
                    

    return all_data

b_data = bar_data(all_fight_data, 'Referees')
df = pd.DataFrame(b_data)

In [244]:
weight = []
for event in all_fight_data:
    for fight in event[0]:
        if fight[2][5] not in weight:
            print(fight[2])
            weight.append(fight[2][5])

[0, 'KO/TKO', '1', '1:17', 'John McCarthy', 'UFC 2 Tournament Title Bout', 1, 'Punches to Head From Mount\n      Submission to Strikes']
[0, 'Submission', '1', '1:31', 'John McCarthy', 'Open Weight Bout', 0, 'Gi Choke From Back Control']
[0, 'KO/TKO', '1', '1:27', 'John McCarthy', 'UFC 3 Tournament Title Bout', 1, 'Punches to Head From Mount\n      Submission to Strikes']
[0, 'Submission', '1', '15:49', 'John McCarthy', 'UFC 4 Tournament Title Bout', 1, 'Triangle Choke From Bottom Guard']
[0, 'Submission', '1', '3:01', 'John McCarthy', 'UFC 5 Tournament Title Bout', 1, 'Keylock From Side Control']
[0, 'Submission', '1', '17:45', 'John McCarthy', 'UFC 6 Tournament Title Bout', 1, 'Rear Naked Choke']
[0, 'Submission', '1', '2:14', 'John McCarthy', 'UFC Superfight Championship Bout', 1, 'Guillotine Choke Standing']
[0, 'KO/TKO', '1', '13:17', 'John McCarthy', 'UFC 7 Tournament Title Bout', 1, 'Kicks to Leg At Distance']
[0, 'Decision - Unanimous', '2', '3:00', 'John McCarthy', "Ultimate U

In [245]:
weight = ["Heavyweight", "Light Heavyweight", "Middleweight",
          "Welterweight", "Lightweight", "Featherweight", "Bantamweight", 
          "Flyweight", "Women's Bantamweight", "Women's Strawweight", 
          "Women's Flyweight", "Women's Featherweight", "Open Weight", 
         "Super Heavyweight", "Catch Weight"]